<a href="https://colab.research.google.com/github/soniamerayogonzalez/knapsack_modelos_programacion_grupo_1/blob/main/knapsack_modelos_programacion_grupo_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knapsack problem
## Definición del Problema: La Mochila (Knapsack Problem)

El **Problema de la Mochila** (*Knapsack Problem*) es un problema fundamental en optimización combinatoria y teoría de la complejidad, clasificado como NP-Hard. Debido a la explosión combinatoria de su espacio de búsqueda ($2^n$), es un candidato ideal para ser resuelto mediante Quantum Annealing en procesadores D-Wave.

### Formulación Clásica

Dado un conjunto de $n$ elementos, donde cada elemento $i$ posee un valor $v_i$ y un peso $w_i$, y una capacidad máxima de peso $W$, el objetivo es seleccionar un subconjunto de elementos tal que se maximice el valor total sin exceder la capacidad.

Definimos un vector binario de decisión $x$, donde $x_i \in \{0, 1\}$.

**Función Objetivo (Maximizar):**
$$\max \sum_{i=1}^{n} v_i x_i$$

**Sujeto a la restricción:**
$$\sum_{i=1}^{n} w_i x_i \le W$$

En definitiva, estamos buscando optimizar el llenado de una mochila eligiendo objetos de un conjunto. Cada objeto tiene un peso y un valor, y queremos maximizar el valor introducido en la mochila sin superar el peso máximo que la mochila puede soportar.


## I. La solución rápida: Constrained Quadratic Model

El problema de la mochila está implementado de forma nativa en la biblioteca de generadores de dimod para D-Wave. A continuación se muestra una implementación del problema con código mínimo, en el que se genera de forma aleatoria una configuración para el problema de la mochila y se resuelve. El ExactCQMSolver resuelve todas las posibles configuraciones, y luego podemos filtrar para elegir la mejor solución (menor energía) que satisface las restricciones (peso máximo).

In [1]:
from dimod.generators import random_knapsack
from dimod import ExactCQMSolver
import pandas as pd
import numpy as np

value_range = (10,30) #Los objetos tendrán un valor entre 10 y 30
weight_range = (10,30) #Los objetos tendrán un peso (coste) entre 10 y 30
tightness_ratio = 0.5 # La mochila no puede llevar todo el peso, aproximadamente el doble de peso en los objetos del que podemos meter en la mochila
num_objects = 10 # Habrá un total de 10 objetos entre los que elegir
cqm = random_knapsack(num_objects, value_range=value_range, weight_range=weight_range, tightness_ratio=tightness_ratio)

sampler = ExactCQMSolver()
sampleset = sampler.sample_cqm(cqm)
df = sampleset.to_pandas_dataframe() #Cada fila del Dataframe es una solución

df_filtrado = df[(df['is_feasible'] == True) & (df['is_satisfied'] == True)] #filtramos las soluciones y nos quedamos solo con las que nos interesan
mejor_solucion = df_filtrado.loc[df_filtrado['energy'].idxmin()] #Tras filtrar, nos quedamos con la mejor (la que menor energía tenga)
print("Solución")
print(mejor_solucion)

Solución
x_0                    1
x_1                    0
x_2                    0
x_3                    1
x_4                    0
x_5                    1
x_6                    1
x_7                    1
x_8                    1
x_9                    0
energy            -137.0
is_feasible         True
is_satisfied        True
num_occurrences        1
Name: 490, dtype: object


## II. Desarrollando la solución
El objetivo de esta asignatura es desarrollar nosotros la solución al problema planteado. Para ello, debemos plantear el problema como un modelo Ising o QUBO.

Por definición, el knapsack problem es un problema de tipo "Binary CQM", es decir, Constrained Quadratic Model de tipo binario. Las variables son binarias (los objetos se eligen o no para estar en la mochila) y además de la formulación de la energía (el valor que se quiere minimizar), se debe introducir una restricción adicional para evitar que se supere el peso máximo de la mochila. Sin esa restricción, la solución siempre sería elegir todos los objetos.

Veamos como se generaría la formulación del problema, partiendo de los objetos, sus pesos y sus valores.

### Una clase para encapsular los problemas knapsack

Vamos a construir una clase para encapsular la definición de un problema de la mochila y su posible solución, y facilitar la legibilidad del resto del código.


In [2]:

class KnapsackItem:
    """Clase que representa un objeto individual dentro de la mochila"""
    def __init__(self, name, weight, value, is_slack=False):
        self.name = name
        self._weight = weight
        self._value = value
        self.is_slack = is_slack # Propiedad para identificar si es holgura

    def weight(self):
        return self._weight

    def value(self):
        return self._value

    def __str__(self):
        return self.name

class ProblemaKnapsack:
    def __init__(self, max_weight=0):
        self.objects = [] # Usaremos objetos KnapsackItem
        self.max_weight = max_weight

    def add_object(self, name, weight, value):
        new_item = KnapsackItem(name, weight, value, is_slack=False)
        self.objects.append(new_item)

    def add_slack_object(self, name, weight):
        """Añade un objeto de holgura (sin valor, marcado como slack)"""
        new_item = KnapsackItem(name, weight, 0, is_slack=True)
        self.objects.append(new_item)

    def print_problem(self, mostrar_holgura=False):
        print("Definición del problema")

        # Filtramos objetos para mostrar usando el atributo is_slack del objeto
        objs_to_show = [o for o in self.objects if mostrar_holgura or not o.is_slack]
        print(tabulate( ([o.name, o.weight(), o.value()] for o in objs_to_show),
                        headers=["Objeto","Peso","Valor"]))
        print("Peso máximo: ", self.max_weight)

    def print_solution(self, solution, mostrar_holgura=False):
        print("Solución")

        # Helper para obtener valor de la solución
        def get_selection_status(item_name):
            if hasattr(solution, 'get'):
                return int(solution.get(item_name, 0))
            return 0 # Si falla por índice o tipo, asumimos 0

        # Preparamos datos tabla
        table_data = []
        total_weight = 0
        total_value = 0

        for o in self.objects:
            selected = get_selection_status(o.name)
            if not o.is_slack:
                total_weight += o.weight() * selected
                total_value += o.value() * selected

            if mostrar_holgura or not o.is_slack:
                table_data.append([o.name, o.weight(), o.value(), selected])

        print(tabulate(table_data, headers=["Objeto","Peso","Valor", "Seleccionado"]))
        print("Peso máximo permitido: ", self.max_weight)
        print("Peso total ocupado (sin holgura): ", total_weight)
        print("Valor total conseguido: ", total_value)


### Generando un problema aleatorio
En primer lugar, generemos un problema aleatorio de modo que definamos directamente la lista de objetos y sus características (peso y valor), así como el peso máximo.

Para facilitar la gestión de estos problemas y sus soluciones, vamos a empaquetarlo todo en una clase

In [3]:
#BANCO DE PRUEBA CAMBIAR EN EL RESTO DE CODIGO
import tabulate
import random
from tabulate import tabulate
import math

def random_knapsack_definition(num_objects, value_range, weight_range, tightness_ratio):
    problema = ProblemaKnapsack()

    total_weight_generated = 0

    for i in range(0, num_objects):
        name = "x_" + str(i)
        w = random.randint(weight_range[0], weight_range[1])
        v = random.randint(value_range[0], value_range[1])

        problema.add_object(name, w, v)
        total_weight_generated += w

    problema.max_weight = math.ceil(tightness_ratio * total_weight_generated)

    return problema

# Generamos el problema
problema = random_knapsack_definition(num_objects, value_range, weight_range, tightness_ratio)
problema.print_problem()

Definición del problema
Objeto      Peso    Valor
--------  ------  -------
x_0           22       23
x_1           13       28
x_2           27       28
x_3           21       11
x_4           21       27
x_5           21       30
x_6           28       28
x_7           10       25
x_8           27       27
x_9           23       24
Peso máximo:  107


### Formulando el problema como un CQM
Debemos formular este problema como
- Una función cuadrática que determina el valor a **minimizar** (el valor de todos los objetos en la mochila, para una determinada solución)
- Una restricción que impide que el peso total supere el máximo permitido

Estas son las que hemos visto anteriormente en la formulación clásica del problema:

**Función Objetivo (minimizar):**
$$ - \sum_{i=1}^{n} v_i x_i$$

**Restricción:**
$$\sum_{i=1}^{n} w_i x_i \le W$$



In [4]:
from dimod import Binary, ConstrainedQuadraticModel

valor_total = 0
peso_total = 0

for o in problema.objects:
    # Creamos la variable binaria usando el nombre del objeto
    v = Binary(o.name)

    valor_total += v * o.value()
    peso_total += v * o.weight()

cqm = ConstrainedQuadraticModel()
cqm.set_objective(-valor_total)
cqm.add_constraint(peso_total <= problema.max_weight)

sampler = ExactCQMSolver()
sampleset = sampler.sample_cqm(cqm)
df = sampleset.to_pandas_dataframe()

df_filtrado = df[(df['is_feasible'] == True) & (df['is_satisfied'] == True)]
mejor_solucion = df_filtrado.loc[df_filtrado['energy'].idxmin()]
print("Solución CQM Manual")
print(mejor_solucion)


Solución CQM Manual
x_0                    0
x_1                    1
x_2                    1
x_3                    0
x_4                    0
x_5                    1
x_6                    1
x_7                    1
x_8                    0
x_9                    0
energy            -139.0
is_feasible         True
is_satisfied        True
num_occurrences        1
Name: 229, dtype: object


### Formulando el problema como un BQM
No siempre podemos recurrir a los CQM e introducir restricciones, ya que cuando usamos ordenadores cuánticos para resolver, puede que estos implementen de forma nativa el modelo Ising, que no permite incorporar restricciones.

Podemos formular este problema directamente como un Binary Quadratic Model, un modelo sin restricciones, también binario. Para ello incorporamos la restricción como una penalización en la función de energía.

Esta restricción se puede implementar como el cuadrado de la diferencia entre el peso total y el peso máximo, usando un parámetro, $\alpha$, como factor de ponderación. El cuadrado nos permite que la restricción no tenga nunca un peso negativo. De este modo, se obligará al modelo a intentar aproximarse a la carga máxima (en ese caso la restricción vale cero).  

**Función Objetivo:**
$$ \underbrace{-\sum_{i=1}^{n} v_i x_i}_{\text{Función objetivo anterior}} + \underbrace{\alpha \left( \sum_{i=1}^{n} w_i x_i - W \right)^2}_{\text{Penalización}}$$

Se puede desarrollar el cuadrado, obteniendo:

$$ -\sum_{i=1}^{n} v_i x_i + \alpha \left( \sum_{i=1}^{n}\sum_{j=1}^{n} w_i w_j x_i x_j  + W^2 -2 W \sum_{i=1}^{n} w_i x_i \right)$$

Cabe destacar que en el doble sumatorio se puede dar el caso de $i=j$. Dicho término no sería cuadrático sino lineal, ya que, al ser $x_i$ binario, $x_i^2 = x_i$. Obtendríamos por tanto

$$ -\sum_{i=1}^{n} v_i x_i + \alpha \left( \sum_{i=1}^{n} w_i^2 x_i^2 + \sum_{i=1}^{n}\sum_{\substack{j=1 \\ j \neq i} }^{n} w_i w_j x_i x_j -2 W \sum_{i=1}^{n} w_i x_i + W^2 \right)$$

Que, simplificando, equivaldría a:

$$ -\sum_{i=1}^{n} v_i x_i + \alpha \left( \sum_{i=1}^{n} w_i^2 x_i + \sum_{i=1}^{n}\sum_{\substack{j=1 \\ j \neq i} }^{n} w_i w_j x_i x_j -2 W \sum_{i=1}^{n} w_i x_i \right)$$

Esta formulación se puede reorganizar para encajar en la formulación estándar de un QUBO, con todos los términos lineales agrupados y los cuadráticos agrupados:

$$ \sum_{i=0}^{n}(-v_i + \alpha w_i^2 -2 \alpha W w_i)x_i + \sum_{i=1}^{n}\sum_{\substack{j=1 \\ j \neq i} }^{n} \alpha w_i w_j x_i x_j$$

#### Límites de esta solución
La implementación de la restricción como un cuadrado de la distancia al peso máximo impone ciertas limitaciones. Este método se suele utilizar para implementar restricciones de igualdad, ya que con un parámetro $\alpha$ suficientemente grande, se forzará la búsqueda de una solución con el peso exacto.

Si bien este método es sencillo de implementar, no es óptimo para restricciones de desigualdad.
- Se pueden aceptar soluciones suboptimas a cambio de llenar la mochila
- En caso de no forzar la igualdad (parámetro $\alpha$ pequeño), se pueden obtener soluciones por encima del peso máximo.

Veamos, en cualquier caso, la aproximación simplificada BQM con restricciones débiles.

In [5]:
from dimod import ExactSolver

qubo_basico = {}
alpha = 1

for o_i in problema.objects:
    # Términos lineales (la diagonal de la matriz QUBO)

    qubo_basico[(o_i.name, o_i.name)] = (
        -o_i.value()
        + alpha * (o_i.weight()**2)
        - 2 * alpha * problema.max_weight * o_i.weight()
    )

  # Términos cuadráticos (los elementos fuera de la diagonal de la matriz QUBO)
    for o_j in problema.objects:
        if o_i.name == o_j.name:
          continue
          # Aplicamos la penalización
        qubo_basico[(o_i.name, o_j.name)] = alpha * o_i.weight() * o_j.weight()

exact_solver = ExactSolver()
solucion_qubo_basico = exact_solver.sample_qubo(qubo_basico).first.sample

problema.print_solution(solucion_qubo_basico)



Solución
Objeto      Peso    Valor    Seleccionado
--------  ------  -------  --------------
x_0           22       23               1
x_1           13       28               1
x_2           27       28               0
x_3           21       11               0
x_4           21       27               1
x_5           21       30               1
x_6           28       28               0
x_7           10       25               1
x_8           27       27               0
x_9           23       24               1
Peso máximo permitido:  107
Peso total ocupado (sin holgura):  110
Valor total conseguido:  157


### Calculando $\alpha$
El parámetro de ponderación (alpha en nuestro código) es clave para obtener soluciones óptimas. Un $\alpha$ demasiado pequeño permitirá incumplir la restricción, mientras que un $\alpha$ demasiado grande hará irrelevante el valor a maximizar (o coste a minimizar).

Debe elegirse un $\alpha$ tal que la penalización por violar la restricción siempre sea mayor que la ganancia obtenida por violarla. Puesto que la ganancia se determina en nuestro caso por el valor de los objetos, un posible $\alpha$ sería la suma de todos los valores de todos los objetos. No obstante, para problemas de gran tamaño podemos obtener un $\alpha$ demasiado grande.

En general, una buena heurística es tomar un $\alpha$ ligeramente superior al valor del mayor objeto.

El código en ese caso sería el siguiente:



In [6]:
qubo_basico = {}

# Calculamos alpha buscando el valor máximo entre los objetos
alpha = max(o.value() for o in problema.objects) * 1.1

for o_i in problema.objects:
    # Término lineal
    qubo_basico[(o_i.name, o_i.name)] = (
        - o_i.value()
        + alpha * (o_i.weight()**2)
        - 2 * alpha * problema.max_weight * o_i.weight()
    )

    for o_j in problema.objects:
        if o_i.name == o_j.name:
          continue
        # Término cuadrático
        qubo_basico[(o_i.name, o_j.name)] = alpha * o_i.weight() * o_j.weight()

solucion_qubo_basico = exact_solver.sample_qubo(qubo_basico).first.sample

problema.print_solution(solucion_qubo_basico)




Solución
Objeto      Peso    Valor    Seleccionado
--------  ------  -------  --------------
x_0           22       23               0
x_1           13       28               0
x_2           27       28               1
x_3           21       11               0
x_4           21       27               1
x_5           21       30               1
x_6           28       28               1
x_7           10       25               1
x_8           27       27               0
x_9           23       24               0
Peso máximo permitido:  107
Peso total ocupado (sin holgura):  107
Valor total conseguido:  138


### BQM Avanzado: restricciones fuertes, soluciones óptimas
Para resolver los problemas propios de la formulación simplificada del BQM, se utiliza la técnica conocida como "Slack Variables" o variables de holgura.

La idea es incluir "objetos" que tienen peso pero no añaden valor y nos permiten llenar el espacio vacío de la mochila. De este modo, para cualquier solución que llene la mochila de forma parcial y tenga un máximo de valor, existirá una solución de igual valor con objetos fantasma (sin valor), que ocupará exactamente la mochila completa. El peso total sería:

$$\sum_{i=1}^{n} w_i x_i + S = W$$

donde $S$ es exactamente el espacio vacío en esta solución.

Como los modelos QUBO solo entienden variables binarias (0 o 1), no podemos introducir $S$ directamente como un entero. Debemos descomponer $S$ en una suma de variables binarias auxiliares, $y_k$, que representan un conjunto de objetos que se pueden usar para llenar la mochila. Para asegurar que tenemos suficientes objetos para llenar la mochila justo hasta su límite, usamos potencias de $2$ desde $0$ hasta $M = \log_2 W$), para poder representar cualquier peso añadido hasta el peso máximo.

$$S = \sum_{k=0}^{M} 2^k y_k$$

Por tanto, la función de energía busca minimizar:

$$\underbrace{-\sum_{i=1}^{n} v_i x_i}_{\text{Valor}} + \alpha \underbrace{\left( \sum_{i=1}^{n} w_i x_i + \sum_{k=0}^{M} 2^k y_k - W \right)^2}_{\text{Restricción (Peso + Holgura - Peso máximo)}}$$

Si bien podríamos expandir el cuadrado anterior, la expresión adquiere cierta complejidad. Optaremos alternativamente por implementarlo añadiendo $M$ objetos, $y_k$, que tendrán peso $2^k$ y valor $0$.

Este método nos garantiza una solución a las dos limitaciones indicadas anteriormente, implementando efectivamente una restricción de desigualdad:

- Permite no llenar la mochila: Si la solución óptima pesa 5kg y la capacidad es 10kg, las variables $y_k$ tomarán el valor 5, haciendo que el término de penalización sea $(5+5-10)^2 = 0$. La energía es mínima y válida.

- Impide exceder el peso (Restricción Fuerte): No existen objetos con pesos negativos. Asumiendo que el valor $\alpha$ es suficientemente grande, cualquier solución que exceda el peso tendrá una penalización mayor que la ventaja de sobrepasar el peso.



In [7]:
import copy

problema_holgura = copy.deepcopy(problema)

# Añadimos variables de holgura
smallest_object = min([o.weight() for o in problema_holgura.objects])
# Necesitamos holgura para compensar el espacio libre máximo, asumiendo que al menos el objeto más ligero
# está incluído. Tenemos que sumar 1 ya que si el peso máximo es justo una potencia de dos
# no cubriríamos todo el espacio (log2(8)=3, 2^0 + 2^1 +2^2 = 7)
k = math.ceil(math.log2(problema_holgura.max_weight - smallest_object + 1))
for k in range(0, k):
    o_name = "y_" + str(k)
    problema_holgura.add_slack_object(o_name, 2**k)

qubo_holgura = {}
alpha = max(o.value() for o in problema_holgura.objects) * 1.1

for o_i in problema_holgura.objects:
    qubo_holgura[(o_i.name, o_i.name)] = (
        - o_i.value()
        + alpha * (o_i.weight()**2)
        - 2 * alpha * problema_holgura.max_weight * o_i.weight()
    )

    for o_j in problema_holgura.objects:
        if o_i.name == o_j.name:
          continue
        qubo_holgura[(o_i.name, o_j.name)] = alpha * o_i.weight() * o_j.weight()

solucion_qubo_holgura = exact_solver.sample_qubo(qubo_holgura).first.sample

print("---- Solución normal (Slack oculta) ----")
problema_holgura.print_solution(solucion_qubo_holgura, mostrar_holgura=False)

print("\n---- Solución con variables de holgura visibles ----")
problema_holgura.print_solution(solucion_qubo_holgura, mostrar_holgura=True)



---- Solución normal (Slack oculta) ----
Solución
Objeto      Peso    Valor    Seleccionado
--------  ------  -------  --------------
x_0           22       23               0
x_1           13       28               1
x_2           27       28               1
x_3           21       11               0
x_4           21       27               0
x_5           21       30               1
x_6           28       28               1
x_7           10       25               1
x_8           27       27               0
x_9           23       24               0
Peso máximo permitido:  107
Peso total ocupado (sin holgura):  99
Valor total conseguido:  139

---- Solución con variables de holgura visibles ----
Solución
Objeto      Peso    Valor    Seleccionado
--------  ------  -------  --------------
x_0           22       23               0
x_1           13       28               1
x_2           27       28               1
x_3           21       11               0
x_4           21       27           

## III. Aplicando el problema de la mochila a optimización de carteras
El problema de la mochila se puede aplicar a optimización de carteras de inversión. El problema se define de forma similar al problema de la mochila:
- Existe un conjunto de posibles inversiones a realizar. Cada una de ellas tiene un retorno esperado ($r_x$) y una cantidad a invertir (coste, $c_x$).
- Hay una restricción: la cantidad máxima a invertir, $C$.

En ese sentido, el problema es análogo al definido hasta ahora, y resulta poco práctico en escenarios reales.

Incorporaremos dos elementos adicionales al problema:

- Minimización del riesgo: Cada inversión posible en un activo financiero tiene un riesgo asociado. Este se puede descomponer en dos aspectos:
  - Riesgo intrínseco: el riesgo del propio activo. Por ejemplo, su volatilidad o varianza.
  - Riesgo compuesto: el riesgo asociado a la inversión conjunta en dos activos financieros. Por ejemplo, si ambos activos están altamente correlacionados (invertir en dos empresas del mismo sector), el riesgo global de la inversión aumenta. Sin embargo, si se invierte en activos inversamente correlacionados, el riesgo global de la inversión se reduce (si un activo baja, el otro subirá para compensarlo).
- Variables no binarias: Hasta ahora hemos trabajado con objetos que podían inclurise o no en la mochila. Este modelo es válido por ejemplo para inversiones en las que se adquiere un activo completo (comprar toda la empresa, o todo lo que está disponible para la venta) o se asume un gasto para obtener un retorno (abrir nuevas sucursales de una empresa). No obstante, para ciertos activos financieros la posible inversión no es binaria (invertir o no invertir) sino que es discreta (comprar un número concreto de acciones). Incorporaremos esta opción al problema de la mochila, reformulándolo como un QUBO.

### Formulación matemática de la minimización del riesgo
El parámetro que mide el riesgo se expresa como una relación (covarianza) entre dos variables (el retorno de dos inversiones distintas), y mide como cambia una respecto a la otra (qué pasa al retorno de una inversión cuando la otra cambia en una dirección concreta). Denominamos a esta covarianza $\sigma_{i j}$:

- Si cuando la invesión $i$ va mejor de lo esperado, la inversion $j$ también va mejor de lo esperado, $\sigma_{i j} > 0$
- Si cuando la invesión $i$ va peor de lo esperado, la inversion $j$  va mejor de lo esperado, $\sigma_{i j} < 0$
- $\sigma_{i i}$ sería la varianza propia del activo, una medición de su volatilidad (riesgo intrínseco).

Por simplicidad, para desarrollar esta fórmula, partiremos de la formulación del problema de la mochila con restricción de igualdad.

$$\underbrace{-\sum_{i=0}^{n} r_i x_i}_{\text{Maximizar Retorno}} + \underbrace{\alpha \left(\sum_{i=0}^{n} c_i x_i - C\right)^2}_{\text{Restricción de Presupuesto}} + \underbrace{\beta \sum_{i=0}^{n}\sum_{j=0}^{n} \sigma_{ij} x_i x_j}_{\text{Minimizar Riesgo}}$$

Como podemos observar, la formulación es idéntica a la del problema de la mochila, pero incorpora un término adicional para el riesgo. Por tanto, con respecto a nuestra solución anterior para el problema de la mochila, solo debemos incorporar estos términos adicionales en función de $\sigma$:

- Términos lineales: $\beta \sigma_{i i} x_i$
- Términos cuadráticos: $\beta \sigma_{i j} x_i x_j$

Añadiendo los mismos a nuestra formulación como QUBO para el problema de la mochila, obtendríamos la solución al problema de optimización de carteras. Podemos añadir estos términos tanto a la formulación con variables de holgura (restricción de desigualdad) como a la formulación estándar (restricción de igualdad).

### Formulación matemática de inversiones no binarias

Para hacer inversiones no binarias, el truco que podemos usar es similar al de las variables de holgura. Podemos incorporar en la lista de variables (posibles inversiones) varias copias de la misma inversión, con distintas cantidades.

Si utilizamos la descomposición binaria del máximo de inversión posible, permitiremos invertir cantidades arbitrarias en cada acción. Por ejemplo, si el presupuesto máximo es $W = 1000$ y una determinada acción, $x_i$, vale 10 euros ($C_i = 100$), podemos añadir múltiplos de la acción hasta el máximo, que serían 100 acciones si solo llenásemos la cartera con este activo.

Formalmente, tomaríamos todas las variables $x_{i,k}$, con coste $c_{i,k} = 2^k c_i$, con retorno $r_{i,k} = k r_i$, siendo $k ∈ [0, log_2{C/c_i}]$  

$\textbf{NOTA}\text{: Tenemos que ver como se "compone" el riesgo de un conjunto de acciones, tanto el riesgo intrínseco como el extrínseco}$


$\textbf{NOTA}\text{: Tenemos que ver si el ROI se expresa como un factor sobre el coste (un porcentaje) o si se expresa en términos absolutos. Según el caso, se compone de un modo u otro y el QUBO se formulará distinto}$



En escenarios reales de optimización de carteras, la decisión no suele ser binaria, sino discreta: se decide cuántas unidades de cada activo adquirir.

Denotamos por:

$$
q_i \in \{0,1,2,\dots,Q_i^{\max}\}
$$

el número de unidades del activo $i$ que se compran.

### Descomposición binaria de las cantidades

Dado que los modelos QUBO solo admiten variables binarias, representamos cada cantidad $q_i$ mediante una descomposición binaria:

$$
q_i = \sum_{k=0}^{K_i} 2^k\,x_{i,k},
\quad x_{i,k}\in\{0,1\}
$$

donde:

$$
K_i = \left\lfloor \log_2(Q_i^{\max}) \right\rfloor,
\qquad
Q_i^{\max} = \left\lfloor \frac{C}{p_i} \right\rfloor
$$

siendo:
- $p_i$ el precio por unidad del activo $i$
- $C$ el presupuesto máximo disponible

Cada variable binaria $x_{i,k}$ representa la compra de un “paquete” de $2^k$ unidades del activo $i$.



### Coste total de la cartera

El coste total de la inversión es:

$$
\text{Coste} = \sum_i p_i q_i
= \sum_i \sum_k (2^k p_i)\,x_{i,k}
$$

Por tanto, cada variable $x_{i,k}$ tiene asociado un coste:

$$
c_{i,k} = 2^k p_i
$$

## Clase para abstraer un problema de optimización de carteras
A continuación desarrollamos una clase que encapsula el problema de optimización de carteras. Almacena todos los activos financieros y sus características (coste, retorno, riesgo) y las características generales del problema (inversión máxima, si se permiten multiples inversiones en el mismo activo, paquetes mínimos de acciones para la inversión, máximo de la cartera que se puede usar con un único activo...)

Además, la clase encapsula la generación de variables binarias para representar múltiplos de las acciones o paquetes de acciones, y abstrae el acceso: según el tipo de problema, se accede a las variables originales o a las variables binarias. También permite incorporar variables de holgura.

Finalmente, la clase encapsula un mecanismo para generar soluciones iterativas. Se puede crear una nueva definición del problema a partir de una solución dada. De este modo, se puede buscar una solución aproximada (usando paquetes de acciones grandes) y luego refinarla con paquetes más pequeños en torno a la solución aproximada que se ha obtenido.

In [8]:
import math
from tabulate import tabulate
import copy

class FinancialAsset:
    """
    Representa un activo financiero con sus propiedades fundamentales.
    Almacena el coste, retorno esperado y riesgo intrínseco.
    """
    def __init__(self, name, price, roi, variance=0):
        self.name = name
        self.price = price       # Coste por unidad
        self.roi = roi           # Ratio de retorno (ej. 1.05)
        self.variance = variance # Varianza individual

    def __str__(self):
        return self.name

class DecisionVariable:
    """
    Representa una variable binaria para el optimizador.

    Puede ser:
    1. Una inversión completa (Modo simple).
    2. Un paquete parcial de acciones (Modo múltiplos).
    3. Una variable de holgura (Slack) para ajustar desigualdades.
    """
    def __init__(self, parent_name, quantity, cost, value, is_slack=False):
        self.parent_name = parent_name
        self.quantity = quantity # Número de unidades del activo (o coste en caso de Slack)
        self.cost = cost         # Coste monetario que ocupa en el presupuesto
        self.value = value       # Retorno esperado
        self.is_slack = is_slack # Identificador de variable de holgura

    def __repr__(self):
        type_str = "Slack" if self.is_slack else "Asset"
        return f"[{type_str}: {self.parent_name}, Cost:{self.cost:.1f}, Val:{self.value:.1f}]"

class PortfolioOptimizationProblem:
    """
    Clase principal para modelar problemas de optimización de cartera.
    Soporta descomposición binaria, variables de holgura y refinamiento iterativo.
    """
    def __init__(self, max_budget, allow_multiples=False, min_package_pct=0.0, max_asset_allocation_pct=1.0, use_slack=False):
        """
        :param max_budget: Presupuesto total (C).
        :param allow_multiples: Si True, permite comprar fracciones/múltiplos mediante descomposición binaria.
        :param min_package_pct: Tamaño mínimo del paquete incremental (% del presupuesto).
        :param max_asset_allocation_pct: Límite máximo global de inversión en un solo activo (% del presupuesto).
        :param use_slack: Si True, añade variables de holgura para convertir desigualdades (<= C) en igualdades (= C).
        """
        self.max_budget = max_budget
        self.allow_multiples = allow_multiples
        self.min_package_pct = min_package_pct
        self.max_asset_allocation_pct = max_asset_allocation_pct
        self.use_slack = use_slack

        self.assets = {}
        self.covariances = {}

        # Diccionario para guardar restricciones específicas por activo (usado en refinamiento)
        # Formato: { 'NombreActivo': {'min_pct': 0.20, 'max_pct': 0.40} }
        self.asset_specific_constraints = {}

        self._decision_variables = []
        self._is_compiled = False

    def add_asset(self, name, price, variance=0, roi_ratio=None, expected_value=None):
        """
        Añade un activo. Permite definir el retorno como ratio o valor absoluto (expected value).
        """
        if price <= 0:
            raise ValueError(f"El precio del activo '{name}' debe ser positivo.")

        if roi_ratio is None and expected_value is None:
            raise ValueError(f"En '{name}', define 'roi_ratio' o 'expected_value'.")

        final_roi = roi_ratio
        if final_roi is None:
            final_roi = expected_value / price

        self.assets[name] = FinancialAsset(name, price, final_roi, variance)
        self._is_compiled = False

    def add_covariance(self, asset1_name, asset2_name, covariance_val):
        """Define el riesgo conjunto entre dos activos."""
        key = tuple(sorted((asset1_name, asset2_name)))
        self.covariances[key] = covariance_val

    def set_asset_constraint(self, name, min_pct, max_pct):
        """
        Define restricciones específicas para un activo. Útil para el refinamiento.
        :param min_pct: Porcentaje mínimo de la cartera que debe representar este activo (Base).
        :param max_pct: Porcentaje máximo permitido.
        """
        self.asset_specific_constraints[name] = {'min': min_pct, 'max': max_pct}
        self._is_compiled = False

    def get_risk_coefficient(self, name1, name2):
        """Devuelve varianza o covarianza según corresponda."""
        if name1 == name2:
            return self.assets[name1].variance if name1 in self.assets else 0.0
        key = tuple(sorted((name1, name2)))
        return self.covariances.get(key, 0.0)

    # -------------------------------------------------------------------------
    # GENERACIÓN DE VARIABLES
    # -------------------------------------------------------------------------

    def build_variables(self):
        """Genera la lista completa de variables de decisión (Activos + Slack)."""
        self._decision_variables = []

        # 1. Variables de Activos
        if not self.allow_multiples:
            self._build_single_mode()
        else:
            self._build_multiples_mode()

        # 2. Variables de Holgura (Slack)
        if self.use_slack:
            self._build_slack_variables()

        self._is_compiled = True
        return self._decision_variables

    def _build_single_mode(self):
        for name, asset in self.assets.items():
            if asset.price <= self.max_budget:
                val = asset.price * asset.roi
                self._decision_variables.append(DecisionVariable(name, 1, asset.price, val))

    def _build_multiples_mode(self):
        for name, asset in self.assets.items():
            # Obtener restricciones (Específicas o Globales)
            constraints = self.asset_specific_constraints.get(name, {})
            min_pct = constraints.get('min', 0.0)
            max_pct = constraints.get('max', self.max_asset_allocation_pct)

            # Límites monetarios
            budget_floor = self.max_budget * min_pct
            budget_ceil = min(self.max_budget, self.max_budget * max_pct)

            # Calcular unidades
            # Unidad base determinada por el paquete mínimo global (granularidad)
            granularity_money = self.max_budget * self.min_package_pct

            if asset.price >= granularity_money:
                base_units = 1
            else:
                base_units = max(1, round(granularity_money / asset.price))

            max_units_total = math.floor(budget_ceil / asset.price)
            min_units_base = math.floor(budget_floor / asset.price)

            if max_units_total == 0:
                continue

            current_units_count = 0

            # --- FASE 1: Variable Base (Suelo) ---
            # Si hay un mínimo exigido (ej. 20%), creamos una variable grande inicial.
            # Nota: Esto no obliga al solver a cogerla, pero permite la estructura solicitada.
            if min_units_base > 0:
                cost = min_units_base * asset.price
                val = cost * asset.roi
                # Variable que representa el "bloque base"
                self._decision_variables.append(DecisionVariable(name, min_units_base, cost, val))
                current_units_count += min_units_base

            # --- FASE 2: Incrementos (Descomposición Binaria) ---
            # Generamos variables para cubrir el espacio entre el Mínimo y el Máximo
            remaining_units_capacity = max_units_total - current_units_count

            if remaining_units_capacity > 0:
                step_units = base_units
                accumulated_step = 0

                # Potencias de 2 escaladas
                while accumulated_step + step_units <= remaining_units_capacity:
                    cost = step_units * asset.price
                    val = cost * asset.roi
                    self._decision_variables.append(DecisionVariable(name, step_units, cost, val))

                    accumulated_step += step_units
                    step_units *= 2

                # Resto exacto para llegar al tope
                remainder = remaining_units_capacity - accumulated_step
                if remainder >= 1: # Si queda algo que comprar
                    cost = remainder * asset.price
                    val = cost * asset.roi
                    self._decision_variables.append(DecisionVariable(name, remainder, cost, val))

    def _build_slack_variables(self):
        """
        Genera variables de holgura mediante descomposición binaria del presupuesto total.
        Nombre: 'SLACK'. ROI: 0. Riesgo: 0.
        """
        # Descomponemos el presupuesto total en potencias de 2 para máxima flexibilidad
        # Usamos 1 como unidad base monetaria (o un valor pequeño según precisión)
        current_val = 1
        remaining_budget = self.max_budget

        while current_val <= remaining_budget:
            self._decision_variables.append(
                DecisionVariable("SLACK", current_val, float(current_val), 0.0, is_slack=True)
            )
            remaining_budget -= current_val
            current_val *= 2

        if remaining_budget > 0:
             self._decision_variables.append(
                DecisionVariable("SLACK", remaining_budget, float(remaining_budget), 0.0, is_slack=True)
            )

    def get_variables(self):
        if not self._is_compiled:
            self.build_variables()
        return self._decision_variables

    # -------------------------------------------------------------------------
    # MÉTODO DE REFINAMIENTO (ZOOM IN)
    # -------------------------------------------------------------------------

    def create_refined_problem(self, previous_solution_vector, zoom_range_pct=0.10, new_granularity_pct=0.01):
        """
        Genera una NUEVA instancia del problema para afinar una solución previa.

        :param previous_solution_vector: Array binario (0/1) de la solución anterior.
        :param zoom_range_pct: Margen porcentual (+/-) alrededor de la solución actual.
        :param new_granularity_pct: Nuevo tamaño de paquete mínimo (más fino).
        :return: Nuevo objeto PortfolioOptimizationProblem.
        """
        current_vars = self.get_variables()

        # 1. Calcular inversión actual por activo
        investments = {} # {asset_name: total_cost}
        for i, selected in enumerate(previous_solution_vector):
            if selected > 0 and not current_vars[i].is_slack:
                name = current_vars[i].parent_name
                investments[name] = investments.get(name, 0.0) + current_vars[i].cost

        # 2. Crear nuevo problema
        # Hereda presupuesto y configuración, pero cambia la granularidad
        new_prob = PortfolioOptimizationProblem(
            max_budget=self.max_budget,
            allow_multiples=True,
            min_package_pct=new_granularity_pct, # Granularidad más fina
            max_asset_allocation_pct=self.max_asset_allocation_pct, # Límite global se mantiene
            use_slack=self.use_slack
        )

        # 3. Copiar Activos y Covarianzas
        for a in self.assets.values():
            new_prob.add_asset(a.name, a.price, a.variance, roi_ratio=a.roi)

        new_prob.covariances = copy.deepcopy(self.covariances)

        # 4. Calcular y aplicar nuevos límites (Zoom)
        for name, current_invested in investments.items():
            current_pct = current_invested / self.max_budget

            # Nuevo suelo: Lo que tenías menos el margen (mínimo 0)
            new_min = max(0.0, current_pct - zoom_range_pct)

            # Nuevo techo: Lo que tenías más el margen (máximo global o 100%)
            new_max = min(self.max_asset_allocation_pct, current_pct + zoom_range_pct)

            # Aplicamos la restricción específica al nuevo problema
            new_prob.set_asset_constraint(name, new_min, new_max)

        return new_prob

    # -------------------------------------------------------------------------
    # VISUALIZACIÓN
    # -------------------------------------------------------------------------

    def print_problem(self):
        print("\n=== Definición del Problema ===")
        print(f"Presupuesto: {self.max_budget}")
        print(f"Modo Múltiplos: {self.allow_multiples} | Holgura: {self.use_slack}")

        table = []
        for a in self.assets.values():
            bounds = self.asset_specific_constraints.get(a.name, None)
            bound_str = f"{bounds['min']*100:.1f}% - {bounds['max']*100:.1f}%" if bounds else "Global"
            table.append([a.name, a.price, a.roi, a.variance, bound_str])

        print(tabulate(table, headers=["Activo", "Precio", "ROI", "Var", "Rango Permitido"]))

        if self.use_slack:
            print("\n* Variables de Holgura activadas (permiten desigualdades)")

    def print_solution(self, solution_vector):
        variables = self.get_variables()

        summary = {}
        total_cost = 0
        total_return = 0
        slack_cost = 0

        for i, selected in enumerate(solution_vector):
            if selected > 0:
                var = variables[i]
                if var.is_slack:
                    slack_cost += var.cost
                    continue

                if var.parent_name not in summary:
                    summary[var.parent_name] = {'units': 0, 'cost': 0.0, 'val': 0.0}

                summary[var.parent_name]['units'] += var.quantity
                summary[var.parent_name]['cost'] += var.cost
                summary[var.parent_name]['val']  += var.value

                total_cost += var.cost
                total_return += var.value

        data = []
        for name, info in summary.items():
            pct = (info['cost'] / self.max_budget) * 100
            data.append([name, info['units'], f"{info['cost']:.2f}", f"{pct:.1f}%", f"{info['val']:.2f}"])

        if self.use_slack and slack_cost > 0:
             data.append(["[HOLGURA]", "-", f"{slack_cost:.2f}", f"{(slack_cost/self.max_budget)*100:.1f}%", "0.00"])

        print("\n=== Solución ===")
        print(tabulate(data, headers=["Activo", "Unidades", "Inv.", "% Cart.", "Retorno"]))
        print(f"Total Invertido (Real): {total_cost:.2f}")
        if self.use_slack:
            print(f"Total Presupuesto Ocupado (con Holgura): {total_cost + slack_cost:.2f} / {self.max_budget}")

In [9]:
import numpy as np
import pandas as pd

# Datos fake para probar

# Activos (identificadores)
assets = ["A", "B", "C"]

# Precio por unidad de cada activo (p_i)
prices = {
    "A": 50,
    "B": 30,
    "C": 20
}

# ROI esperado por activo (porcentaje esperado)
roi = {
    "A": 0.08,   # 8%
    "B": 0.05,   # 5%
    "C": 0.10    # 10%
}

# Presupuesto total disponible
C = 200

# Matriz de covarianzas (riesgo)
# Orden: A, B, C
Sigma = np.array([
    [0.04,  0.01, -0.02],
    [0.01,  0.03,  0.00],
    [-0.02, 0.00,  0.05]
])

# Parámetros del modelo
alpha = 0.125   # penalización del presupuesto
beta  = 1.0    # aversión al riesgo

In [10]:
import math

#Crear variables binarias x_{i,k}

lots = []  # aquí guardaremos cada variable binaria con su info (coste, retorno, unidades)

for asset in assets:
    p = prices[asset]
    qmax = int(C // p)  # máximo de unidades si gastamos todo el presupuesto en este activo

    if qmax <= 0:
        continue

    K = int(math.floor(math.log2(qmax)))  # bits necesarios (0..K)

    for k in range(K + 1):
        units = 2**k                     # unidades representadas por este bit
        var = f"x_{asset}_{k}"           # nombre de variable binaria
        cost = units * p                 # coste asociado a activar esta variable
        ret_unit = roi[asset] * p        # retorno esperado por unidad (ROI * precio)
        ret = units * ret_unit           # retorno esperado de este lote

        lots.append({
            "var": var,
            "asset": asset,
            "k": k,
            "units": units,
            "price": p,
            "cost": cost,
            "ret": ret
        })

lots_df = pd.DataFrame(lots)

print("Variables binarias generadas (lotes):")
display(lots_df)

print("\nResumen por activo:")
display(lots_df.groupby("asset")[["units","cost","ret"]].sum())

Variables binarias generadas (lotes):


,var,asset,k,units,price,cost,ret
0,x_A_0,A,0,1,50,50,4.0
1,x_A_1,A,1,2,50,100,8.0
2,x_A_2,A,2,4,50,200,16.0
3,x_B_0,B,0,1,30,30,1.5
4,x_B_1,B,1,2,30,60,3.0
5,x_B_2,B,2,4,30,120,6.0
6,x_C_0,C,0,1,20,20,2.0
7,x_C_1,C,1,2,20,40,4.0
8,x_C_2,C,2,4,20,80,8.0
9,x_C_3,C,3,8,20,160,16.0



Resumen por activo:


,units,cost,ret
asset,,,
A,7,350,28.0
B,7,210,10.5
C,15,300,30.0


### Retorno esperado

Existen dos interpretaciones habituales del retorno esperado.


#### Retorno absoluto por unidad

Si el retorno esperado por unidad del activo $i$ es $r_i$ (en unidades monetarias):

$$
\text{Retorno} = \sum_i r_i q_i
= \sum_i \sum_k (2^k r_i)\,x_{i,k}
$$

#### Retorno relativo (ROI)

Si el retorno se expresa como un porcentaje $\rho_i$ (ROI), el retorno monetario esperado por unidad es $\rho_i p_i$:

$$
\text{Retorno} = \sum_i (\rho_i p_i) q_i
= \sum_i \sum_k (2^k \rho_i p_i)\,x_{i,k}
$$

En ambos casos, el término de retorno se introduce en el QUBO con signo negativo:

$$
-\sum_{i,k} R_{i,k}\,x_{i,k}
$$

In [11]:
from collections import defaultdict

# Mapa para localizar índices de activos en Sigma
asset_index = {a: i for i, a in enumerate(assets)}

# Diccionario QUBO (default 0.0)
Q = defaultdict(float)

#Término de retorno (lineal):  - sum R_v x_v
# En QUBO, el término lineal va en la diagonal: Q[(v,v)] += coef
for row in lots:
    v = row["var"]
    Q[(v, v)] += -row["ret"]

### Riesgo de la cartera con inversiones no binarias

El riesgo de una cartera se modela mediante la matriz de covarianzas $\sigma_{ij}$.

Para cantidades no binarias, el riesgo total es:

$$
\text{Riesgo} = \sum_{i=1}^n \sum_{j=1}^n \sigma_{ij}\,q_i q_j
$$

Sustituyendo la descomposición binaria:

$$
q_i q_j
= \left(\sum_k 2^k x_{i,k}\right)\left(\sum_l 2^l x_{j,l}\right)
= \sum_k \sum_l 2^{k+l} x_{i,k} x_{j,l}
$$

Por tanto:

$$
\text{Riesgo}
= \sum_{i,j}\sum_{k,l}\sigma_{ij}\,2^{k+l}\,x_{i,k}x_{j,l}
$$

Este término penaliza tanto el riesgo intrínseco ($i=j$) como el riesgo conjunto entre activos correlacionados ($i\neq j$).

Introducimos un parámetro $\beta$ para controlar la aversión al riesgo.

In [12]:
# Término de riesgo (cuadrático): beta * sum_{i,j} sigma_ij q_i q_j ---
# Con q_i = sum_k units * x_{i,k}, el término induce acoplos entre variables
# Añadimos:
#   beta * sigma_{a,b} * units_u * units_v  a Q[(u,v)]
# Para u==v, se suma en la diagonal (es válido en QUBO).
for u in lots:
    a = u["asset"]
    ia = asset_index[a]
    for v in lots:
        b = v["asset"]
        ib = asset_index[b]
        coef = beta * Sigma[ia, ib] * (u["units"] * v["units"])

        # Para evitar duplicar (u,v) y (v,u), metemos solo si u_var <= v_var
        uvar, vvar = u["var"], v["var"]
        if uvar <= vvar:
            Q[(uvar, vvar)] += coef #Se tienen en cuenta ambas contribuciones

### Restricción de presupuesto

#### Restricción de igualdad

Si se desea invertir exactamente el presupuesto disponible:

$$
\alpha\left(\sum_{i,k} c_{i,k}x_{i,k} - C\right)^2
$$

#### Restricción de desigualdad (con holgura)

Si se permite no invertir todo el presupuesto:

$$
\sum_{i,k} c_{i,k}x_{i,k} + S = C,
\qquad
S = \sum_{m=0}^{M}2^m y_m
$$

donde $y_m \in \{0,1\}$ son variables binarias auxiliares sin retorno asociado.

In [13]:
# Presupuesto (penalización): alpha * (sum cost_v x_v - C)^2
# Expansión:
# alpha * [ (sum c_v x_v)^2 - 2C (sum c_v x_v) + C^2 ]
# El término C^2 es constante => se ignora
#
# (sum c_v x_v)^2 = sum c_v^2 x_v + 2 * sum_{u<v} c_u c_v x_u x_v
#
# Por tanto:
# Diagonal:   + alpha * c_v^2  - 2*alpha*C*c_v
# Offdiag:    + 2*alpha*c_u*c_v   (u<v)
for i, u in enumerate(lots):
    uvar = u["var"]
    cu = u["cost"]

    # diagonal: alpha*c^2 - 2*alpha*C*c
    Q[(uvar, uvar)] += alpha * (cu ** 2) - 2.0 * alpha * C * cu

    # off-diagonal: 2*alpha*c_u*c_v
    for j in range(i + 1, len(lots)):
        v = lots[j]
        vvar = v["var"]
        cv = v["cost"]
        key = (uvar, vvar) if uvar <= vvar else (vvar, uvar)
        Q[key] += 2.0 * alpha * cu * cv

### Función de energía QUBO final

La función de energía a minimizar queda:

$$
E(x) =
-\sum_{i,k} R_{i,k}x_{i,k}
+ \beta\sum_{i,j}\sum_{k,l}\sigma_{ij}2^{k+l}x_{i,k}x_{j,l}
+ \alpha\left(\sum_{i,k} c_{i,k}x_{i,k} - C\right)^2
$$

Esta formulación permite resolver el problema de optimización de carteras con inversiones no binarias dentro del marco QUBO.

In [14]:
# Convertimos Q a dict normal (opcional, pero suele ser más cómodo)
Q = dict(Q)

print(f"Número de variables binarias (lotes): {len({k[0] for k in Q.keys()} | {k[1] for k in Q.keys()})}")
print(f"Número de términos QUBO: {len(Q)}")

# Muestra unos cuantos términos para verificar
sample_items = list(Q.items())[:10]
print("\nEjemplo de coeficientes QUBO (primeros 10):")
for (a, b), w in sample_items:
    print(f"Q[({a}, {b})] = {w:.4f}")

Número de variables binarias (lotes): 10
Número de términos QUBO: 55

Ejemplo de coeficientes QUBO (primeros 10):
Q[(x_A_0, x_A_0)] = -2191.4600
Q[(x_A_1, x_A_1)] = -3757.8400
Q[(x_A_2, x_A_2)] = -5015.3600
Q[(x_B_0, x_B_0)] = -1388.9700
Q[(x_B_1, x_B_1)] = -2552.8800
Q[(x_B_2, x_B_2)] = -4205.5200
Q[(x_C_0, x_C_0)] = -951.9500
Q[(x_C_1, x_C_1)] = -1803.8000
Q[(x_C_2, x_C_2)] = -3207.2000
Q[(x_C_3, x_C_3)] = -4812.8000


In [15]:
from dimod import ExactSolver

# Resolver QUBO y analizar resultado

# 1) Resolver el QUBO (exacto: válido para pocas variables)
solver = ExactSolver()
sampleset = solver.sample_qubo(Q)
best = sampleset.first
solution = best.sample
energy = best.energy

print("Mejor energía encontrada:", energy)
print("Variables activas (valor 1):")
active_vars = [v for v, val in solution.items() if val == 1]
print(active_vars)

# 2) Reconstruir unidades por activo a partir de los lotes
units_by_asset = {a: 0 for a in assets}
cost_by_asset  = {a: 0.0 for a in assets}
ret_by_asset   = {a: 0.0 for a in assets}

# Crear un índice rápido: var -> info del lote
lot_by_var = {row["var"]: row for row in lots}

for v in active_vars:
    info = lot_by_var[v]
    a = info["asset"]
    units_by_asset[a] += info["units"]
    cost_by_asset[a]  += info["cost"]
    ret_by_asset[a]   += info["ret"]

total_cost = sum(cost_by_asset.values())
total_ret  = sum(ret_by_asset.values())

# 3) Calcular riesgo (Markowitz) usando q_i (unidades) y Sigma
q = np.array([units_by_asset[a] for a in assets], dtype=float)
risk = float(q.T @ Sigma @ q)  # varianza (sin raíz). Si queremos volatilidad: np.sqrt(risk) si risk>=0

# 4) Mostrar resumen tabular
summary_df = pd.DataFrame({
    "asset": assets,
    "units": [units_by_asset[a] for a in assets],
    "cost":  [cost_by_asset[a] for a in assets],
    "ret":   [ret_by_asset[a] for a in assets],
})

print("\n=== Resumen cartera (mejor solución) ===")
display(summary_df)

print(f"Presupuesto C = {C}")
print(f"Coste total   = {total_cost:.2f}")
print(f"Retorno esp.  = {total_ret:.2f}")
print(f"Riesgo (q^TΣq)= {risk:.6f}")

# 5) Comprobación rápida del presupuesto
if abs(total_cost - C) < 1e-6:
    print("Presupuesto EXACTO (igualdad).")
elif total_cost <= C:
    print("Coste <= presupuesto (si querías desigualdad, esto sería válido).")
else:
    print("Coste > presupuesto: alpha probablemente es demasiado pequeño o falta holgura.")

Mejor energía encontrada: -5016.99
Variables activas (valor 1):
['x_A_1', 'x_C_0', 'x_C_2']

=== Resumen cartera (mejor solución) ===


,asset,units,cost,ret
0,A,2,100.0,8.0
1,B,0,0.0,0.0
2,C,5,100.0,10.0


Presupuesto C = 200
Coste total   = 200.00
Retorno esp.  = 18.00
Riesgo (q^TΣq)= 1.010000
Presupuesto EXACTO (igualdad).


##Preparación de Datos para Optimización de Cartera QUBO

El objetivo de esta fase es transformar los precios históricos brutos en los tres parámetros clave requeridos por nuestro modelo QUBO (Optimización Cuadrática Binaria Sin Restricciones).

Para ello se utiliza un enfoque de Backtesting, donde la optimización se realiza con datos de un período pasado (In-Sample) para validar su efectividad en un período futuro (Out-of-Sample).

Los tres inputs esenciales que obtendremos de la historia de precios son:



1.   Retorno Esperado ($r_i$): El beneficio esperado de cada activo.
2.   Coste ($c_i$): El precio de compra en el momento de la optimización.
3.   Matriz de Covarianza ($\sigma_{ij}$): El componente de riesgo que mide cómo se mueven los activos entre sí.

El siguiente código Python descarga y calcula estos parámetros utilizando un período de entrenamiento fijo (2023-2025) para simular una decisión de inversión tomada a principios de 2025.

In [16]:
import yfinance as yf
import numpy as np

# --- 1. Definición de Parámetros de Entrada para Backtesting ---

# Tickers de activos (Sectores y Geografías)
# ^GSPC: S&P 500 (Índice USA) | RY: Royal Bank of Canada (Finanzas Canadá)
# NEE: NextEra Energy (Utilities/Energía USA) | DIS: Walt Disney Co (Entretenimiento)
# BABA: Alibaba (Tech China) | SNEJF: Sony Group Corp (Tech/Electro Japón)

TICKERS = ['^GSPC', 'RY', 'NEE', 'DIS', 'BABA', 'SNEJF']
START_DATE = '2023-01-01'
END_DATE = '2025-01-01'
INTERVALO_ANALISIS = '1d' # Diario


# --- 2. Descarga de Datos Históricos (Datasheet Crudo) ---

datos_crudos = yf.download(TICKERS,
                           start=START_DATE,
                           end=END_DATE,
                           interval=INTERVALO_ANALISIS,
                           auto_adjust=True)

precios = datos_crudos['Close'].dropna()


# --- 3. Derivación de Parámetros Clave para el Modelo (a 2025-01-01) ---

## PARÁMETRO 1: Coste (c_i)

# El coste c_i es el precio más reciente antes de la fecha de inversión (END_DATE).
costes_ci = precios.iloc[-1].rename('Coste ($c_i$)')


## PARÁMETRO 2: Retornos Diarios y Retorno Esperado (r_i)

# Calcular los retornos diarios logarítmicos
retornos = np.log(precios / precios.shift(1)).dropna()

# Multiplicamos por 252 (días de trading en un año) para anualizar.
roi = retornos.mean() * 252
roi.rename('Retorno Esperado ($r_i$ Anualizado)', inplace=True)


## PARÁMETRO 3: Matriz de Covarianza (Sigma_ij)

# La Matriz de Covarianza (Sigma) se calcula a partir de los retornos, anualizada
sigma_matrix = retornos.cov() * 252

print(f"\n- **Fecha de Optimización (Entrada del Modelo):** {END_DATE}")
print(f"\n- **r_i (Retorno Esperado):**\n{roi}")
print(f"\n- **c_i (Coste/Precio):**\n{costes_ci}")
print(f"\n- **Matriz Sigma (Covarianza):**\n{sigma_matrix}")

[*********************100%***********************]  6 of 6 completed


- **Fecha de Optimización (Entrada del Modelo):** 2025-01-01

- **r_i (Retorno Esperado):**
Ticker
BABA    -0.023409
DIS      0.119037
NEE     -0.050245
RY       0.167152
SNEJF    0.183437
^GSPC    0.216539
Name: Retorno Esperado ($r_i$ Anualizado), dtype: float64

- **c_i (Coste/Precio):**
Ticker
BABA       83.380714
DIS       110.877174
NEE        69.521393
RY        117.415466
SNEJF      21.183086
^GSPC    5881.629883
Name: Coste ($c_i$), dtype: float64

- **Matriz Sigma (Covarianza):**
Ticker      BABA       DIS       NEE        RY     SNEJF     ^GSPC
Ticker                                                            
BABA    0.152255  0.014846  0.010667  0.018923  0.016533  0.014204
DIS     0.014846  0.068022  0.009189  0.013235  0.004690  0.013574
NEE     0.010667  0.009189  0.073529  0.015555  0.005086  0.008027
RY      0.018923  0.013235  0.015555  0.027834  0.005026  0.011657
SNEJF   0.016533  0.004690  0.005086  0.005026  0.149083  0.008122
^GSPC   0.014204  0.013574  0.00802

 ## Evaluación del Rendimiento de la Cartera (Backtesting Post-Inversión)

 Una vez que el modelo de optimización QUBO haya utilizado los datos históricos (hasta 2025-01-01) para seleccionar la cartera óptima de activos ($x_i$), el siguiente paso crucial es el Backtesting.

 Este código tiene como objetivo simular la inversión real midiendo el rendimiento acumulado que esos activos obtuvieron en el mercado en el tiempo posterior a la decisión.

 Al analizar el rendimiento en intervalos de 15 días, 1 mes, 2 meses, etc., podemos confirmar la efectividad de la optimización. Si la cartera seleccionada por el modelo supera consistentemente al mercado o a carteras aleatorias, se valida la calidad de la decisión.

 El código descarga los precios desde el día después de la inversión y calcula la tasa de crecimiento de cada activo en los puntos temporales definidos.

In [17]:
from datetime import date
from dateutil.relativedelta import relativedelta

# Fecha de inicio del Backtesting (1  día después de la inversión)
START_BACKTEST = pd.to_datetime(END_DATE) + pd.Timedelta(days=1)

END_BACKTEST_MAX = pd.to_datetime('2025-07-02')

# Definición de los puntos de tiempo para el análisis
TIMEFRAMES = {
    '15 Días': START_BACKTEST + pd.Timedelta(days=15),
    'Mes 1': START_BACKTEST + relativedelta(months=1),
    'Mes 2': START_BACKTEST + relativedelta(months=2),
    'Mes 3': START_BACKTEST + relativedelta(months=3),
    'Mes 4': START_BACKTEST + relativedelta(months=4),
    'Mes 5': START_BACKTEST + relativedelta(months=5),
    'Mes 6': START_BACKTEST + relativedelta(months=6),
}


# 1. Descarga de Datos (Out-of-Sample)

datos_crudos_out = yf.download(
    TICKERS,
    start=START_BACKTEST.strftime('%Y-%m-%d'),
    end=END_BACKTEST_MAX.strftime('%Y-%m-%d'),
    interval=INTERVALO_ANALISIS,
    auto_adjust=True
)

precios_out = datos_crudos_out['Close'].dropna()

# Precio de Cierre el día de la inversión (P_inicial)
P_INICIAL = precios_out.iloc[0]


# 2. Cálculo del Rendimiento por Intervalo

resultados_rendimiento = {}

for label, end_date in TIMEFRAMES.items():
    # Encuentra la fila con la fecha más cercana a end_date
    P_FINAL_SERIE = precios_out.loc[precios_out.index <= end_date].iloc[-1]

    # Rendimiento Acumulado = ((P_final / P_inicial) - 1)
    rendimiento_pct = ((P_FINAL_SERIE / P_INICIAL) - 1)

    resultados_rendimiento[label] = rendimiento_pct.round(2)

# 3. Presentación de Resultados


print("\n Comportamiento del Mercado (Rendimiento Acumulado)")
print(f"Inversión Base (P_inicial): {START_BACKTEST.date()}")
print("-" * 60)


df_resultados = pd.DataFrame(resultados_rendimiento)
df_resultados = df_resultados.transpose()
df_resultados.index.name = 'Intervalo'

print(df_resultados)


[*********************100%***********************]  6 of 6 completed


 Comportamiento del Mercado (Rendimiento Acumulado)
Inversión Base (P_inicial): 2025-01-02
------------------------------------------------------------
Ticker     BABA   DIS   NEE    RY  SNEJF  ^GSPC
Intervalo                                      
15 Días    0.00 -0.03 -0.01  0.01  -0.03   0.02
Mes 1      0.16  0.02 -0.00  0.03   0.04   0.03
Mes 2      0.56  0.03 -0.01 -0.00   0.27   0.01
Mes 3      0.53 -0.12 -0.01 -0.03   0.22  -0.03
Mes 4      0.48 -0.17 -0.06  0.03   0.23  -0.03
Mes 5      0.35  0.02 -0.00  0.09   0.26   0.01
Mes 6      0.36  0.12  0.04  0.12   0.21   0.06


In [18]:
import pandas as pd
import numpy as np
from dimod import ExactSolver
from collections import defaultdict
import math

# -----------------------------------------------------------------------------
# 1. CONFIGURACIÓN: Mapeo de Datos Reales al Modelo
# -----------------------------------------------------------------------------

# Usamos los datos obtenidos de yfinance en las celdas anteriores
assets = TICKERS  # Lista de nombres ['^GSPC', 'RY', ...]

# Convertimos las Series de Pandas a Diccionarios para acceso rápido
prices = costes_ci.to_dict()  # { 'BABA': 83.38, ... }
roi_dict = roi.to_dict()      # { 'BABA': -0.023, ... } (Renombramos para no confundir)
Sigma = sigma_matrix.values   # Convertimos el DataFrame de covarianza a matriz Numpy

# --- Parámetros de Inversión ---
# IMPORTANTE: Definimos un presupuesto C.
# Nota: Como usamos ExactSolver, mantén el presupuesto moderado para no
# generar demasiadas variables binarias (bits), o el solver tardará mucho.
C = 1000.0 

# Hiperparámetros del modelo
alpha = 0.5   # Penalización por desviarse del presupuesto (fuerza a gastar C)
beta  = 1.0   # Aversión al riesgo (peso del término de covarianza)

print(f"Configurando optimización para Presupuesto: ${C}")
print(f"Activos disponibles: {len(assets)}")

# -----------------------------------------------------------------------------
# 2. GENERACIÓN DE VARIABLES BINARIAS (Lotes)
# -----------------------------------------------------------------------------
# Replicamos la lógica de descomposición binaria con los datos reales

lots = []
for asset in assets:
    p = prices[asset]
    
    # Si el precio es mayor que el presupuesto, no podemos comprar ni una unidad
    if p > C:
        continue
        
    qmax = int(C // p)  # Máximo de acciones posibles
    
    if qmax <= 0:
        continue

    # Calcular cuántos bits (potencias de 2) necesitamos
    K = int(math.floor(math.log2(qmax))) 

    for k in range(K + 1):
        units = 2**k
        # Aseguramos no pasarnos del qmax total en la última potencia si fuera necesario, 
        # pero la descomposición estándar 2^k suele ser suficiente para aproximar.
        
        var_name = f"x_{asset}_{k}"
        cost = units * p
        
        # El retorno esperado total de este lote
        # Retorno unitario = ROI anualizado * Precio
        ret_expected = (roi_dict[asset] * p) * units
        
        lots.append({
            "var": var_name,
            "asset": asset,
            "k": k,
            "units": units,
            "price": p,
            "cost": cost,
            "ret": ret_expected
        })

print(f"Total de variables binarias generadas: {len(lots)}")
if len(lots) > 25:
    print("¡ADVERTENCIA! Más de 25 variables puede ser muy lento para ExactSolver.")

# -----------------------------------------------------------------------------
# 3. CONSTRUCCIÓN DE LA MATRIZ QUBO (Q)
# -----------------------------------------------------------------------------
Q = defaultdict(float)
asset_index = {a: i for i, a in enumerate(assets)}

# A) Término de Retorno (Maximizar retorno -> Minimizar negativo)
for row in lots:
    v = row["var"]
    Q[(v, v)] += -row["ret"]

# B) Término de Riesgo (Minimizar Varianza)
for u in lots:
    a_name = u["asset"]
    ia = asset_index[a_name]
    for v in lots:
        b_name = v["asset"]
        ib = asset_index[b_name]
        
        # Covarianza entre activo A y activo B
        cov_val = Sigma[ia, ib]
        
        # Coeficiente: beta * sigma * unidades_u * unidades_v
        coef = beta * cov_val * (u["units"] * v["units"])
        
        uvar, vvar = u["var"], v["var"]
        
        # Sumar al término correspondiente en la matriz triangular superior
        if uvar == vvar:
            Q[(uvar, uvar)] += coef
        elif uvar < vvar: # Solo una vez para pares distintos
            Q[(uvar, vvar)] += 2 * coef # x2 porque la matriz es simétrica
            
# C) Término de Presupuesto (Penalización alpha * (coste_total - C)^2)
# Expansión: alpha * [ sum(c_i^2 x_i) + 2*sum(c_i c_j x_i x_j) - 2*C*sum(c_i x_i) ]
for i, u in enumerate(lots):
    uvar = u["var"]
    cu = u["cost"]
    
    # Diagonal: alpha * cost^2 - 2 * alpha * C * cost
    Q[(uvar, uvar)] += alpha * (cu**2) - 2.0 * alpha * C * cu
    
    # Fuera de diagonal: 2 * alpha * cost_u * cost_v
    for j in range(i + 1, len(lots)):
        v = lots[j]
        vvar = v["var"]
        cv = v["cost"]
        
        # Key ordenada
        key = (uvar, vvar) if uvar <= vvar else (vvar, uvar)
        Q[key] += 2.0 * alpha * cu * cv

# -----------------------------------------------------------------------------
# 4. RESOLUCIÓN Y VISUALIZACIÓN DE RESULTADOS
# -----------------------------------------------------------------------------

# Resolver QUBO
print("Resolviendo QUBO (esto puede tardar unos segundos)...")
solver = ExactSolver()
sampleset = solver.sample_qubo(Q)
best = sampleset.first
solution = best.sample
energy = best.energy

print("Mejor energía encontrada:", energy)

# Filtrar variables activas
active_vars = [v for v, val in solution.items() if val == 1]

# Reconstruir cartera
units_by_asset = {a: 0 for a in assets}
cost_by_asset  = {a: 0.0 for a in assets}
ret_by_asset   = {a: 0.0 for a in assets}
lot_by_var = {row["var"]: row for row in lots}

for v in active_vars:
    info = lot_by_var[v]
    a = info["asset"]
    units_by_asset[a] += info["units"]
    cost_by_asset[a]  += info["cost"]
    ret_by_asset[a]   += info["ret"]

total_cost = sum(cost_by_asset.values())
total_ret  = sum(ret_by_asset.values())

# Calcular riesgo resultante
q_vec = np.array([units_by_asset[a] for a in assets], dtype=float)
final_risk = float(q_vec.T @ Sigma @ q_vec)

# Crear DataFrame de resumen
summary_df = pd.DataFrame({
    "Ticker": assets,
    "Unidades": [units_by_asset[a] for a in assets],
    "Precio Unit.": [prices[a] for a in assets],
    "Inversión ($)": [cost_by_asset[a] for a in assets],
    "Retorno Esp. ($)": [ret_by_asset[a] for a in assets],
})

# Formateo visual
print("\n=== CARTERA ÓPTIMA (QUBO) BASADA EN DATOS REALES ===")
display(summary_df[summary_df['Unidades'] > 0]) # Solo mostramos lo que compramos

print("-" * 40)
print(f"Presupuesto Objetivo: ${C}")
print(f"Inversión Total:      ${total_cost:.2f}")
print(f"Retorno Esperado:     ${total_ret:.2f}")
print(f"Riesgo (Varianza):    {final_risk:.6f}")

if abs(total_cost - C) < (C * 0.05):
    print("Estado: Presupuesto utilizado correctamente.")
else:
    print("Estado: Inversión baja. Intenta aumentar 'alpha' o revisar precios vs presupuesto.")

Configurando optimización para Presupuesto: $1000.0
Activos disponibles: 6
Total de variables binarias generadas: 22
Resolviendo QUBO (esto puede tardar unos segundos)...
Mejor energía encontrada: -500163.62486249354

=== CARTERA ÓPTIMA (QUBO) BASADA EN DATOS REALES ===


,Ticker,Unidades,Precio Unit.,Inversión ($),Retorno Esp. ($)
1,RY,6,117.415466,704.492798,117.757640
5,SNEJF,14,21.183086,296.563210,54.400811


----------------------------------------
Presupuesto Objetivo: $1000.0
Inversión Total:      $1001.06
Retorno Esperado:     $172.16
Riesgo (Varianza):    7.976013
Estado: Presupuesto utilizado correctamente.


## Optimizando el uso de variables

Con la formulación actual, para cada posible acción necesitamos $log_2{C/c_i}$ variables, a las que hay que añadir las variables de holgura. Esto hace que la complejidad del problema crezca rápidamente.

Existen diversas técnicas para optimizar el uso de variables:
- No usar variables de holgura: Puesto que podemos hacer inversiones de cantidades arbitrarias de cada activo, si no incluímos variables de holgura, el modelo intentará maximizar las inversiones para usar todo el presupuesto. Esto es una buena aproximación, ya que en general el inversor querrá invertir todo su presupuesto. Así podemos evitar las variables de holgura.
- Establecer límites al porcentaje de cada activo: es habitual que no se quiera invertir más de una cierta cantidad en un único activo, con objeto de garantizar la diversificación. Esto es algo que el modelo implementará automáticamente al ser la varianza (riesgo intrínseco) normalmente mayor que la covarianza (riesgo extrínseco). Pero podemos forzar una restricción que nos ayude a limitar el número de variables. En lugar de tomar el tamaño de la cartera y crear variables para poder llenarla con cada activo financiero, tomaremos, por ejemplo, un 20% del tamaño de la cartera (si establecemos que nunca queremos que una inversión ocupe más del 20%). De este modo, serán necesarias menos variables adicionales.
- Crear paquetes de acciones: podemos representar un activo financiero como un múltiplo de la acción individual, y calcular proporcionalmente su coste, retorno y riesgo. De este modo, al ser el coste mayor respecto al coste total, necesitaremos menos variables para poder llenar la mochila con este activo. Esta opción se puede implementar con paquetes arbitrariamente grandes, si bien reduce la capacidad del modelo de ajustar de forma precisa (es decir, no encontraremos el mínimo local real).
- Iterar con paquetes de acciones de distinto tamaño: Podemos hacer una iteración inicial con paquetes de acciones relativamente grandes (por ejemplo el 5%-10% del tamaño máximo de la cartera) para obtener una solución aproximada. Luego, podemos redefinir el problema en torno a cada solución. Para una determinada acción, tendríamos un paquete inicial que vendría dado, por ejemplo, por el 0.8 del valor de la solución aproximada, y añadiríamos paquetes adicionales para poder alcanzar el 1.2 de ese valor. Esto nos permite "ajustar" la solución para buscar el mínimo local, sin tener que usar todas las variables a la vez, haciendo varias iteraciones.